In [1]:
import numpy as np
import pandas as pd 
import seaborn as sns 
import matplotlib.pyplot as plt 
import warnings 
import folium
from folium.plugins import HeatMap
warnings.filterwarnings('ignore')

In [2]:
chicago = pd.read_csv('datasets//Chicago_Crimes.csv', encoding='ISO-8859-1')

In [3]:
chicago

,ID,Case Number,Date,Block,IUCR,Primary Type,Description,Location Description,Arrest,Domestic,...,Ward,Community Area,FBI Code,X Coordinate,Y Coordinate,Year,Updated On,Latitude,Longitude,Location
0,13439321,JH237424,04/14/2024 12:00:00 AM,040XX S PRAIRIE AVE,0890,THEFT,FROM BUILDING,APARTMENT,False,False,...,3,38.0,06,1178707.0,1878256.0,2024,12/21/2024 03:40:46 PM,41.821236,-87.619921,"(41.821236024, -87.619920712)"
1,13437420,JH234779,04/14/2024 12:00:00 AM,023XX W CERMAK RD,2825,OTHER OFFENSE,HARASSMENT BY TELEPHONE,COMMERCIAL / BUSINESS OFFICE,False,False,...,25,31.0,26,1161210.0,1889347.0,2024,12/21/2024 03:40:46 PM,41.852052,-87.683801,"(41.852051675, -87.683800849)"
2,13428676,JH224478,04/14/2024 12:00:00 AM,043XX W LE MOYNE ST,0917,MOTOR VEHICLE THEFT,"CYCLE, SCOOTER, BIKE WITH VIN",STREET,False,False,...,36,23.0,07,1146960.0,1909501.0,2024,12/21/2024 03:40:46 PM,41.907640,-87.735587,"(41.907640473, -87.735587478)"
3,13429357,JH225293,04/14/2024 12:00:00 AM,039XX W ADAMS ST,143A,WEAPONS VIOLATION,UNLAWFUL POSSESSION - HANDGUN,STREET,True,False,...,28,26.0,15,1150158.0,1898721.0,2024,12/21/2024 03:40:46 PM,41.877997,-87.724121,"(41.877997275, -87.724120826)"
4,13430098,JH226395,04/14/2024 12:00:00 AM,011XX W 112TH PL,0890,THEFT,FROM BUILDING,RESIDENCE,False,False,...,21,75.0,06,1170856.0,1830157.0,2024,12/21/2024 03:40:46 PM,41.689421,-87.650123,"(41.6894214, -87.650123247)"
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
249118,13805239,JJ217509,04/12/2025 12:00:00 AM,029XX W LOGAN BLVD,2826,OTHER OFFENSE,HARASSMENT BY ELECTRONIC MEANS,APARTMENT,False,False,...,1,22.0,26,1156478.0,1917149.0,2025,04/19/2025 03:41:24 PM,41.928440,-87.700416,"(41.928439867, -87.700415972)"
249119,13804023,JJ215813,04/12/2025 12:00:00 AM,094XX S HARVARD AVE,0430,BATTERY,AGGRAVATED - OTHER DANGEROUS WEAPON,STREET,False,False,...,9,49.0,04B,1175694.0,1842631.0,2025,04/19/2025 03:41:24 PM,41.723545,-87.632040,"(41.723545182, -87.632039508)"
249120,13803926,JJ215943,04/12/2025 12:00:00 AM,084XX S VINCENNES AVE,0486,BATTERY,DOMESTIC BATTERY SIMPLE,APARTMENT,False,True,...,21,71.0,08B,1173850.0,1848976.0,2025,04/19/2025 03:41:24 PM,41.740998,-87.638606,"(41.74099774, -87.638606337)"
249121,13803475,JJ215338,04/12/2025 12:00:00 AM,050XX S ABERDEEN ST,0530,ASSAULT,AGGRAVATED - OTHER DANGEROUS WEAPON,STREET,True,False,...,20,61.0,04A,1169838.0,1871348.0,2025,04/19/2025 03:41:24 PM,41.802477,-87.652657,"(41.802477219, -87.652657244)"


In [4]:
chicago['Primary Type'].unique()

array(['THEFT', 'OTHER OFFENSE', 'MOTOR VEHICLE THEFT',
       'WEAPONS VIOLATION', 'BATTERY', 'ASSAULT',
       'CRIMINAL SEXUAL ASSAULT', 'CRIMINAL TRESPASS', 'CRIMINAL DAMAGE',
       'DECEPTIVE PRACTICE', 'SEX OFFENSE', 'ROBBERY', 'NARCOTICS',
       'HOMICIDE', 'INTERFERENCE WITH PUBLIC OFFICER', 'BURGLARY',
       'ARSON', 'OFFENSE INVOLVING CHILDREN', 'INTIMIDATION',
       'PUBLIC PEACE VIOLATION', 'CONCEALED CARRY LICENSE VIOLATION',
       'KIDNAPPING', 'STALKING', 'LIQUOR LAW VIOLATION', 'PROSTITUTION',
       'GAMBLING', 'OBSCENITY', 'PUBLIC INDECENCY', 'HUMAN TRAFFICKING',
       'OTHER NARCOTIC VIOLATION', 'NON-CRIMINAL'], dtype=object)

In [5]:
chicago.columns

Index(['ID', 'Case Number', 'Date', 'Block', 'IUCR', 'Primary Type',
       'Description', 'Location Description', 'Arrest', 'Domestic', 'Beat',
       'District', 'Ward', 'Community Area', 'FBI Code', 'X Coordinate',
       'Y Coordinate', 'Year', 'Updated On', 'Latitude', 'Longitude',
       'Location'],
      dtype='object')

In [6]:
chicago['Date'] = pd.to_datetime(chicago['Date'], errors='coerce')

<h1  >THEFT ANALYSIS</h1>


In [7]:
theft_analysis1 = chicago[chicago['Primary Type'] == 'THEFT']
theft_analysis2 = theft_analysis1[theft_analysis1['Location Description'] == 'SIDEWALK'] 
theft_analysis3 = theft_analysis2.groupby(['Latitude', 'Longitude']).size().reset_index(name= 'incident_count')
theft_analysis3["normalize_data"] = theft_analysis3['incident_count'] - theft_analysis3['incident_count'].min() / \
                                theft_analysis3['incident_count'].max() - theft_analysis3['incident_count'] 
theft_heatmap = theft_analysis3[['Latitude', 'Longitude', 'incident_count']].values.tolist() 
theft_map =  folium.Map(location = [42, -87], zoom_start = 7) 
HeatMap(theft_heatmap).add_to(theft_map) 
theft_map.save('theft.html')
print('data save to theft.html') 
theft_map

data save to theft.html


<h1  >OTHER OFFENSE ANALYSIS</h1>

In [8]:
other_offense_analysis1 = chicago[chicago['Primary Type'] == 'OTHER OFFENSE']
other_offense_analysis2 = other_offense_analysis1[other_offense_analysis1['Year'] >= 2024]
other_offense_analysis3 = other_offense_analysis2[other_offense_analysis2['Beat'] >= 1000] 
otheroffense_final = other_offense_analysis3.groupby(['Latitude', 'Longitude']).size().reset_index(name= 'incident_count')
otheroffense_final 
otheroffense_final["normalize_data"] = otheroffense_final['incident_count'] - otheroffense_final['incident_count'].min() / \
                                otheroffense_final['incident_count'].max() - otheroffense_final['incident_count'] 
otheroffense_heatmap = otheroffense_final[['Latitude', 'Longitude', 'incident_count']].values.tolist() 
otheroffense_map =  folium.Map(location = [42,-87], zoom_start = 7)
HeatMap(otheroffense_heatmap).add_to(otheroffense_map) 
otheroffense_map.save('other_offense.html')
print('data save to other_offense.html') 
otheroffense_map

data save to other_offense.html


<h1  >MOTOR VEHICLE THEFT ANALYSIS</h1>

In [9]:
motor_teft_analysis1 = chicago[chicago['Primary Type'] == 'MOTOR VEHICLE THEFT' ]
motor_teft_analysis2 = motor_teft_analysis1[motor_teft_analysis1['Year'] >= 2024]
motor_teft_analysis3 = motor_teft_analysis2[motor_teft_analysis2['Location Description'] == 'STREET'] 
motor_final = motor_teft_analysis3.groupby(['Latitude', 'Longitude']).size().reset_index(name= 'incident_count')
motor_final 
motor_final["normalize_data"] = motor_final['incident_count'] - motor_final['incident_count'].min() / \
                                motor_final['incident_count'].max() - motor_final['incident_count']
motor_heatmap = motor_final[['Latitude','Longitude','incident_count']].values.tolist() 
motor_map =  folium.Map(location = [42,-87], zoom_start = 7) 
HeatMap(motor_heatmap).add_to(motor_map) 
motor_map.save('motor.html')
print('data save to motor.html') 
motor_map

data save to motor.html



<h1  >WEAPONS VIOLATION ANALYSIS</h1>

In [10]:
weapon_violations1 = chicago[chicago['Primary Type'] == 'WEAPONS VIOLATION']
weapon_violations2 = weapon_violations1[weapon_violations1['District'] >= 10]
weapon_violations3 = weapon_violations2[weapon_violations2['Arrest'] == True]  
weapons_final = weapon_violations3.groupby(['Latitude', 'Longitude']).size().reset_index(name= 'incident_count')
weapons_final 
weapons_final["normalize_data"] = weapons_final['incident_count'] - weapons_final['incident_count'].min() / \
                                weapons_final['incident_count'].max() - weapons_final['incident_count']
weapons_heatmap = weapons_final[['Latitude','Longitude','incident_count']].values.tolist() 
weapons_map =  folium.Map(location = [42,-87], zoom_start = 7) 
HeatMap(weapons_heatmap).add_to(weapons_map) 
weapons_map.save('weapons.html')
print('data save to weapons.html') 
weapons_map

data save to weapons.html



<h1  >BATTERY ANALYSIS</h1>

In [11]:
battery_analysis1 = chicago[chicago['Primary Type'] == 'BATTERY']
battery_analysis2 = battery_analysis1[battery_analysis1['Year'] >= 2024]
battery_analysis3 = battery_analysis2[battery_analysis2['Location Description'] == 'SMALL RETAIL STORE']  
battery_final = battery_analysis3.groupby(['Latitude', 'Longitude']).size().reset_index(name= 'incident_count')
battery_final 
battery_final["normalize_data"] = battery_final['incident_count'] - battery_final['incident_count'].min() / \
                                battery_final['incident_count'].max() - battery_final['incident_count']
battery_heatmap = battery_final[['Latitude','Longitude','incident_count']].values.tolist() 
battery_map =  folium.Map(location = [42,-87], zoom_start = 7) 
HeatMap(battery_heatmap).add_to(battery_map) 
battery_map.save('battery.html')
print('data save to battery.html') 
battery_map

data save to battery.html



<h1  >ASSAULT ANALYSIS</h1>

In [12]:
assault_analysis1 = chicago[chicago['Primary Type'] == 'ASSAULT'] 
assault_analysis2 = assault_analysis1[assault_analysis1['Year'] == 2025]
assault_analysis3 = assault_analysis2[assault_analysis2['Arrest'] == True]  
assault_final = assault_analysis3.groupby(['Latitude', 'Longitude']).size().reset_index(name= 'incident_count')
assault_heatmap = assault_final[['Latitude','Longitude','incident_count']].values.tolist() 
assault_map =  folium.Map(location = [42,-87], zoom_start = 7) 
HeatMap(assault_heatmap).add_to(assault_map) 
assault_map.save('assault.html')
print('data save to assault.html') 
assault_map

data save to assault.html



<h1  >CRIMINAL SEXUAL ASSAULT ANALYSIS</h1>

In [13]:
criminal_sexual1= chicago[chicago['Primary Type'] == 'CRIMINAL SEXUAL ASSAULT']
criminal_sexual2 = criminal_sexual1[criminal_sexual1['Year'] >=  2024] 
criminal_sexual3 = criminal_sexual2[criminal_sexual2['Location Description'] == 'RESIDENCE']  
criminalassault_final = criminal_sexual3.groupby(['Latitude', 'Longitude']).size().reset_index(name= 'incident_count')  
criminalassault_heatmap = criminalassault_final[['Latitude','Longitude','incident_count']].values.tolist() 
criminalassault_map =  folium.Map(location = [42,-87], zoom_start = 7) 
HeatMap(criminalassault_heatmap).add_to(criminalassault_map) 
criminalassault_map.save('criminal_assault.html')
print('data save to criminal_assault.html') 
criminalassault_map

data save to criminal_assault.html



<h1  >CRIMINAL TRESPASS ANALYSIS</h1>

In [14]:
criminal_trespass1 = chicago[chicago['Primary Type'] == 'CRIMINAL TRESPASS'] 
criminal_trespass2 = criminal_trespass1[criminal_trespass1['Location Description'] == 'RESIDENCE']
criminal_trespass3 = criminal_trespass2[criminal_trespass2['Year'] >= 2024] 
trespass_final = criminal_trespass3.groupby(['Latitude', 'Longitude']).size().reset_index(name= 'incident_count')
trespass_final 
trespass_heatmap = trespass_final[['Latitude','Longitude','incident_count']].values.tolist() 
trespass_map =  folium.Map(location = [42,-87], zoom_start = 7) 
HeatMap(trespass_heatmap).add_to(trespass_map) 
trespass_map.save('trespass.html')
print('data save to trespass.html') 
trespass_map

data save to trespass.html



<h1  >CRIMINAL DAMAGE ANALYSIS</h1>


In [15]:
criminal_damage1 = chicago[chicago['Primary Type'] == 'CRIMINAL DAMAGE']
criminal_damage2 = criminal_damage1[criminal_damage1['Description'] == 'TO VEHICLE' ]
criminal_damage3 = criminal_damage2[criminal_damage2['Year'] >= 2025]
criminal_damage4 = criminal_damage3[criminal_damage3['Arrest'] == False] 
damage_final = criminal_damage4.groupby(['Latitude', 'Longitude']).size().reset_index(name= 'incident_count')
damage_heatmap = damage_final[['Latitude','Longitude','incident_count']].values.tolist() 
damage_map =  folium.Map(location = [42,-87], zoom_start = 7) 
HeatMap(damage_heatmap).add_to(damage_map) 
damage_map.save('damage.html')
print('data save to damage.html') 
damage_map

data save to damage.html



<h1  >DECEPTIVE PRACTICE ANALYSIS</h1>


In [16]:
deceptive1 = chicago[chicago['Primary Type'] == 'DECEPTIVE PRACTICE']
deceptive2 = deceptive1[deceptive1['Description'] == 'COUNTERFEITING DOCUMENT']
deceptive3 = deceptive2[deceptive2['Year'] >= 2024] 
deceptive_final = deceptive3.groupby(['Latitude', 'Longitude']).size().reset_index(name= 'incident_count')
deceptive_heatmap = deceptive_final[['Latitude','Longitude','incident_count']].values.tolist() 
deceptive_map =  folium.Map(location = [42,-87], zoom_start = 7) 
HeatMap(deceptive_heatmap).add_to(deceptive_map) 
deceptive_map.save('deceptive.html')
print('data save to deceptive.html') 
deceptive_map

data save to deceptive.html



<h1  >SEX OFFENSE ANALYSIS</h1>


In [17]:
sex1 = chicago[chicago['Primary Type'] == 'SEX OFFENSE']
sex2 = sex1[sex1['Location Description'] == 'RESIDENCE']
sex3 = sex2[sex2['Year'] >= 2024] 
sex_final = sex3.groupby(['Latitude', 'Longitude']).size().reset_index(name= 'incident_count')
sex_heatmap = sex_final[['Latitude','Longitude','incident_count']].values.tolist() 
sex_map =  folium.Map(location = [42,-87], zoom_start = 7) 
HeatMap(sex_heatmap).add_to(sex_map) 
sex_map.save('sex.html')
print('data save to sex.html') 
sex_map

data save to sex.html



<h1  >ROBBERY ANALYSIS</h1>


In [18]:
robbery1 = chicago[chicago['Primary Type'] == 'ROBBERY']
robbery2 = robbery1[robbery1['Location Description'] == 'SMALL RETAIL STORE']
robbery3 = robbery2[robbery2['Year'] >= 2024] 
robbery_final = robbery3.groupby(['Latitude', 'Longitude']).size().reset_index(name= 'incident_count')
robbery_heatmap = robbery_final[['Latitude','Longitude','incident_count']].values.tolist() 
robbery_map =  folium.Map(location = [42,-87], zoom_start = 7) 
HeatMap(robbery_heatmap).add_to(robbery_map) 
robbery_map.save('robbery.html')
print('data save to robbery.html') 
robbery_map

data save to robbery.html



<h1  >NARCOTICS ANALYSIS</h1>


In [19]:
narcotics1= chicago[chicago['Primary Type'] == 'NARCOTICS']
narcotics2 = narcotics1[narcotics1['Description'] == 'FOUND SUSPECT NARCOTICS']
narcotics3 = narcotics2[narcotics2['Year'] >= 2024] 
narcotics_final = narcotics3 = narcotics2[narcotics2['Year'] >= 2024].groupby(['Latitude', 'Longitude']).size().reset_index(name= 'incident_count')
narcotics_final 
narcotics_heatmap = narcotics_final[['Latitude','Longitude','incident_count']].values.tolist() 
narcotics_map =  folium.Map(location = [42,-87], zoom_start = 7) 
HeatMap(narcotics_heatmap).add_to(narcotics_map) 
narcotics_map.save('narcotics.html')
print('data save to narcotics.html') 
narcotics_map

data save to narcotics.html


<h1  >HOMICIDE ANALYSIS</h1>

In [20]:
homicide1 = chicago[chicago['Primary Type'] == 'HOMICIDE'] 
homicide2 = homicide1[homicide1['Location Description'] == 'HOUSE']
homicide3 = homicide2[homicide2['Year'] >= 2024] 
homicide_final = homicide3.groupby(['Latitude', 'Longitude']).size().reset_index(name= 'incident_count')
homicide_final 
homicide_heatmap = homicide_final[['Latitude','Longitude','incident_count']].values.tolist() 
homicide_map =  folium.Map(location = [42,-87], zoom_start = 7) 
HeatMap(homicide_heatmap).add_to(homicide_map) 
homicide_map.save('homicide.html')
print('data save to homicide.html') 
homicide_map

data save to homicide.html


<h1  >INTERFERENCE WITH PUBLIC OFFICER ANALYSIS</h1>

In [21]:
interference_filter = chicago[chicago['Primary Type'] == 'INTERFERENCE WITH PUBLIC OFFICER']
interference_filter2 = interference_filter[interference_filter['Description'] == 'ESCAPE']
interference_filter3 = interference_filter2[interference_filter2['Arrest'] == False]  
interference_final = interference_filter3.groupby(['Latitude', 'Longitude']).size().reset_index(name= 'incident_count') 
interference_final 
interference_heatmap = interference_final[['Latitude','Longitude','incident_count']].values.tolist() 
interference_map =  folium.Map(location = [42,-87], zoom_start = 7) 
HeatMap(interference_heatmap).add_to(interference_map) 
interference_map.save('interference.html')
print('data save to interference.html') 
interference_map

data save to interference.html


<h1  >BURGLARY ANALYSIS</h1>

In [22]:
burglary1 = chicago[chicago['Primary Type'] == 'BURGLARY'] 
burglary2 = burglary1[burglary1['Location Description'] == 'RESIDENCE'] 
burglary3 = burglary2[burglary2['Year'] >= 2025]  
burglary_final = burglary3.groupby(['Latitude', 'Longitude']).size().reset_index(name= 'incident_count')
burglary_final 
burglary_heatmap = burglary_final[['Latitude','Longitude','incident_count']].values.tolist() 
burglary_map =  folium.Map(location = [42,-87], zoom_start = 7) 
HeatMap(burglary_heatmap).add_to(burglary_map) 
burglary_map.save('burglary.html')
print('data save to burglary.html') 
burglary_map

data save to burglary.html


<h1  >ARSON ANALYSIS</h1>

In [23]:
arson1 = chicago[chicago['Primary Type'] == 'ARSON'] 
arson2 = arson1[arson1['Location Description'] == 'COMMERCIAL / BUSINESS OFFICE'] 
arson_final = arson2.groupby(['Latitude', 'Longitude']).size().reset_index(name= 'incident_count')
arson_heatmap = arson_final[['Latitude','Longitude','incident_count']].values.tolist() 
arson_map =  folium.Map(location = [42,-87], zoom_start = 7) 
HeatMap(arson_heatmap).add_to(arson_map) 
arson_map.save('arson.html')
print('data save to arson.html') 
arson_map

data save to arson.html


<h1  >OFFENSE INVOLVING CHILDREN ANALYSIS</h1>

In [24]:
children1 = chicago[chicago['Primary Type'] == 'OFFENSE INVOLVING CHILDREN']
children2 = children1[children1['Location Description'] == 'SCHOOL - PUBLIC GROUNDS'] 
children_final = children2.groupby(['Latitude', 'Longitude']).size().reset_index(name= 'incident_count') 
children_final 
children_heatmap = children_final[['Latitude','Longitude','incident_count']].values.tolist() 
children_map =  folium.Map(location = [42,-87], zoom_start = 7) 
HeatMap(children_heatmap).add_to(children_map) 
children_map.save('children.html')
print('data save to children.html') 
children_map

data save to children.html


<h1  >INTIMIDATION ANALYSIS</h1>

In [25]:
intimidation1 = chicago[chicago['Primary Type'] == 'INTIMIDATION'] 
intimidation2 = intimidation1[intimidation1['Location Description'] == 'STREET']  
intimidation_final = intimidation2.groupby(['Latitude', 'Longitude']).size().reset_index(name= 'incident_count')
intimidation_final 
intimidation_heatmap = intimidation_final[['Latitude','Longitude','incident_count']].values.tolist() 
intimidation_map =  folium.Map(location = [42,-87], zoom_start = 7) 
HeatMap(intimidation_heatmap).add_to(intimidation_map) 
intimidation_map.save('intimidation.html')
print('data save to intimidation.html') 
intimidation_map

data save to intimidation.html


<h1  >PUBLIC PEACE VIOLATION ANALYSIS</h1>

In [26]:
publicpeace1 = chicago[chicago['Primary Type'] == 'PUBLIC PEACE VIOLATION']
publicpeace2 = publicpeace1[publicpeace1['Description'] == 'BOMB THREAT'] 
publicpeace_final = publicpeace2.groupby(['Latitude', 'Longitude']).size().reset_index(name= 'incident_count')
publicpeace_final 
publicpeace_heatmap = publicpeace_final[['Latitude','Longitude','incident_count']].values.tolist() 
publicpeace_map =  folium.Map(location = [42,-87], zoom_start = 7) 
HeatMap(publicpeace_heatmap).add_to(publicpeace_map) 
publicpeace_map.save('publicpeace.html')
print('data save to publicpeace.html') 
publicpeace_map

data save to publicpeace.html


<h1  >CONCEALED CARRY LICENSE VIOLATION ANALYSIS</h1>

In [27]:
concealed1 = chicago[chicago['Primary Type'] == 'CONCEALED CARRY LICENSE VIOLATION']
concealed2 = concealed1[concealed1['Location Description'] == 'AIRPORT TERMINAL UPPER LEVEL - SECURE AREA']
concealed3 = concealed2[concealed2['Year'] >= 2024]  
concealed_final = concealed3.groupby(['Latitude', 'Longitude']).size().reset_index(name= 'incident_count')
concealed_final 
concealed_heatmap = concealed_final[['Latitude','Longitude','incident_count']].values.tolist() 
concealed_map =  folium.Map(location = [42,-87], zoom_start = 7) 
HeatMap(concealed_heatmap).add_to(concealed_map) 
concealed_map.save('concealed_license.html')
print('data save to concealed_license.html') 
concealed_map

data save to concealed_license.html


<h1  >KIDNAPPING ANALYSIS</h1>

In [28]:
kidnapping1 = chicago[chicago['Primary Type'] == 'KIDNAPPING']
kidnapping2 = kidnapping1[kidnapping1['Description'] == 'CHILD ABDUCTION / STRANGER']
kidnapping_final = kidnapping2.groupby(['Latitude', 'Longitude']).size().reset_index(name= 'incident_count')
kidnapping_heatmap = kidnapping_final[['Latitude','Longitude','incident_count']].values.tolist()
kidnapping_map =  folium.Map(location = [42,-87], zoom_start = 7)
HeatMap(kidnapping_heatmap).add_to(kidnapping_map)
kidnapping_map.save('kidnapping.html')
print('data save to kidnapping.html')
kidnapping_map

data save to kidnapping.html


<h1  >STALKING ANALYSIS</h1>

In [29]:
stalking_filter = chicago[chicago['Primary Type'] == 'STALKING'] 
stalking_filter2 = stalking_filter[stalking_filter['Location Description'] == 'RESIDENCE']
stalking_filter3 = stalking_filter2[stalking_filter2['Description'] == 'CYBERSTALKING'] 
stalking_final = stalking_filter3.groupby(['Latitude', 'Longitude']).size().reset_index(name= 'incident_count')
stalking_heatmap = stalking_final[['Latitude','Longitude','incident_count']].values.tolist()
stalking_map =  folium.Map(location = [42,-87], zoom_start = 7)
HeatMap(stalking_heatmap).add_to(stalking_map)
stalking_map.save('stalking.html')
print('data save to stalking.html')
stalking_map

data save to stalking.html


<h1  >SLIQUOR LAW VIOLATION ANALYSIS</h1>

In [30]:
liquor1= chicago[chicago['Primary Type'] == 'LIQUOR LAW VIOLATION']
liquor2 = liquor1[liquor1['Location Description'] == 'TAVERN / LIQUOR STORE' ]
liquor3 = liquor2[liquor2['Description'] == 'LIQUOR LICENSE VIOLATION' ]
liquor_final = liquor3.groupby(['Latitude', 'Longitude']).size().reset_index(name= 'incident_count')
liquor_heatmap = liquor_final[['Latitude','Longitude','incident_count']].values.tolist()
liquor_map =  folium.Map(location = [42,-87], zoom_start = 7)
HeatMap(liquor_heatmap).add_to(liquor_map)
liquor_map.save('liquor.html')
print('data save to liquor.html')
liquor_map

data save to liquor.html


<h1  >PROSTITUTION ANALYSIS</h1>

In [31]:
prostitution1 = chicago[chicago['Primary Type'] == 'PROSTITUTION']
prostitution2 = prostitution1[prostitution1['Location Description'] == 'STREET'] 
prostitution3 = prostitution2[prostitution2['Year'] >= 2024]
prostitution_final = prostitution3.groupby(['Latitude', 'Longitude']).size().reset_index(name= 'incident_count')
prostitution_heatmap = prostitution_final[['Latitude','Longitude','incident_count']].values.tolist()
prostitution_map =  folium.Map(location = [42,-87], zoom_start = 7)
HeatMap(prostitution_heatmap).add_to(prostitution_map)
prostitution_map.save('prostitution.html')
print('data save to prostitution.html')
prostitution_map

data save to prostitution.html


<h1  >GAMBLING ANALYSIS</h1>

In [32]:
gambling1 = chicago[chicago['Primary Type'] == 'GAMBLING']
gambling2 = gambling1[gambling1['Year'] >= 2024]
gambling_final = gambling2.groupby(['Latitude', 'Longitude']).size().reset_index(name= 'incident_count')
gambling_heatmap = gambling_final[['Latitude','Longitude','incident_count']].values.tolist()
gambling_map =  folium.Map(location = [42,-87], zoom_start = 7)
HeatMap(gambling_heatmap).add_to(gambling_map)
gambling_map.save('gambling.html')
print('data save to gambling.html')
gambling_map

data save to gambling.html


<h1  >OBSCENITY ANALYSIS</h1>

In [33]:
obscenity1 = chicago[chicago['Primary Type'] == 'OBSCENITY'] 
obscenity2 = obscenity1[obscenity1['Year'] >= 2024] 
obscenity_final = obscenity2.groupby(['Latitude', 'Longitude']).size().reset_index(name= 'incident_count')
obscenity_heatmap = obscenity_final[['Latitude','Longitude','incident_count']].values.tolist()
obscenity_map =  folium.Map(location = [42,-87], zoom_start = 7)
HeatMap(obscenity_heatmap).add_to(obscenity_map)
obscenity_map.save('obscenity.html')
print('data save to obscenity.html')
obscenity_map

data save to obscenity.html


<h1  >PUBLIC INDECENCY ANALYSIS</h1>

In [34]:
public_indecency1 = chicago[chicago['Primary Type'] == 'PUBLIC INDECENCY']
public_indecency2 = public_indecency1[public_indecency1['Year'] >= 2024]
indecency_final = public_indecency2.groupby(['Latitude', 'Longitude']).size().reset_index(name= 'incident_count')
indecency_heatmap = indecency_final[['Latitude','Longitude','incident_count']].values.tolist()
indecency_map =  folium.Map(location = [42,-87], zoom_start = 7)
HeatMap(indecency_heatmap).add_to(indecency_map)
indecency_map.save('indecency.html')
print('data save to indecency.html')
indecency_map

data save to indecency.html


<h1  >HUMAN TRAFFICKING ANALYSIS</h1>

In [35]:
humantraffic1 = chicago[chicago['Primary Type'] == 'HUMAN TRAFFICKING']
humantraffic2 = humantraffic1[humantraffic1['Year'] >= 2024]
humantraffic_final = humantraffic2.groupby(['Latitude', 'Longitude']).size().reset_index(name= 'incident_count')
humantraffic_heatmap = humantraffic_final[['Latitude','Longitude','incident_count']].values.tolist()
humantraffic_map =  folium.Map(location = [42,-87], zoom_start = 7)
HeatMap(humantraffic_heatmap).add_to(humantraffic_map)
humantraffic_map.save('human_traffic.html')
print('data save to human_traffic.html')
humantraffic_map

data save to human_traffic.html


<h1  >OTHER NARCOTIC VIOLATION ANALYSIS</h1>

In [36]:
other_narcotic1 = chicago[chicago['Primary Type'] == 'OTHER NARCOTIC VIOLATION']
other_narcotic2 = other_narcotic1[other_narcotic1['Year'] >= 2024]
narcotic_final = other_narcotic2.groupby(['Latitude', 'Longitude']).size().reset_index(name= 'incident_count')
narcotic_heatmap = narcotic_final[['Latitude','Longitude','incident_count']].values.tolist()
narcotic_map =  folium.Map(location = [42,-87], zoom_start = 7)
HeatMap(narcotic_heatmap).add_to(narcotic_map)
narcotic_map.save('narcotic.html')
print('data save to narcotic.html')
narcotic_map

data save to narcotic.html


<h1  >NON-CRIMINAL ANALYSIS</h1>

In [37]:
noncriminal_analysis1 = chicago[chicago['Primary Type'] == 'NON-CRIMINAL'] 
noncriminal_analysis2 = noncriminal_analysis1[noncriminal_analysis1['Year'] >= 2024]
noncriminal_final = noncriminal_analysis2.groupby(['Latitude', 'Longitude']).size().reset_index(name= 'incident_count') 
noncriminal_heatmap = noncriminal_final[['Latitude','Longitude','incident_count']].values.tolist()
noncriminal_map =  folium.Map(location = [42,-87], zoom_start = 7)
HeatMap(noncriminal_heatmap).add_to(noncriminal_map)
noncriminal_map.save('non_criminal.html')
print('data save to non_criminal.html')
noncriminal_map

data save to non_criminal.html
